<a href="https://colab.research.google.com/github/LeonardoLuca/AskYou/blob/main/rag_chatbot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab com RAG e Gemini Integrados

In [ ]:
# CÉLULA 1: INSTALAÇÕES

# Instala todas as bibliotecas necessárias de uma vez
!pip install -q langchain langchain-google-genai sentence-transformers faiss-cpu torch google-colab


In [ ]:
# CÉLULA 2: IMPORTS E CONFIGURAÇÃO DA API KEY

import os
import sys
import json
import time
import numpy as np
import faiss
from google.colab import userdata, drive
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from sentence_transformers import SentenceTransformer

# --- Configuração da Chave de API do Google ---
GOOGLE_API_KEY = None
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("AVISO: Chave 'GOOGLE_API_KEY' encontrada nos Secrets, mas está vazia.", file=sys.stderr)
        GOOGLE_API_KEY = None # Garante que é None se estiver vazia
    else:
        print("Chave de API do Google carregada com sucesso!")
except userdata.SecretNotFoundError:
    print("ERRO: Secret 'GOOGLE_API_KEY' não encontrado.", file=sys.stderr)
    print("Por favor, adicione sua chave de API do Google AI Studio aos Secrets do Colab.", file=sys.stderr)
except Exception as e:
    print(f"ERRO ao buscar a chave de API: {e}", file=sys.stderr)

# Limpa a variável de ambiente se existir (boa prática)
if 'GOOGLE_API_KEY' in os.environ:
    del os.environ['GOOGLE_API_KEY']

Chave de API do Google carregada com sucesso!


In [ ]:
# CÉLULA 3: INICIALIZAÇÃO DOS MODELOS (GEMINI E EMBEDDING)

# --- Inicializa o Modelo Gemini ---
chat_model = None
if GOOGLE_API_KEY:
    try:
        # Use o modelo Gemini desejado (Pro ou Flash)
        chat_model = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro-latest", # Ou "gemini-1.5-flash-latest" para mais rápido/barato
            google_api_key=GOOGLE_API_KEY,
            temperature=0.2, # Reduz a criatividade para respostas mais factuais Variabilidade
            # safety_settings=... # Adicione configurações de segurança se necessário
        )
        print(f"Modelo Gemini '{chat_model.model}' instanciado com sucesso.")
    except Exception as e:
        print(f"ERRO ao instanciar o modelo ChatGoogleGenerativeAI: {e}", file=sys.stderr)
else:
    print("AVISO: Modelo Gemini não será instanciado pois a chave de API não foi carregada.", file=sys.stderr)

# --- Inicializa o Modelo de Embedding ---
embedder_model = None
try:
    print("Carregando modelo de embedding (SentenceTransformer)...")
    start_time = time.time()
    # Modelo leve e eficaz para semântica geral
    embedder_model = SentenceTransformer('all-MiniLM-L6-v2')
    print(f"Modelo de embedding carregado em {time.time() - start_time:.2f} segundos.")
except Exception as e:
    print(f"ERRO ao carregar o modelo de embedding: {e}", file=sys.stderr)

Modelo Gemini 'models/gemini-1.5-pro-latest' instanciado com sucesso.
Carregando modelo de embedding (SentenceTransformer)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Modelo de embedding carregado em 4.00 segundos.


In [ ]:
# ===================================================
#             CÉLULA 4: MONTAGEM DO DRIVE E CARREGAMENTO/INDEXAÇÃO DO CV (COM IndexFlatIP)
# ===================================================

# --- Monta o Google Drive ---
# (Como estava antes)
try:
    drive.mount("/content/drive", force_remount=True)
    print("Google Drive montado com sucesso em /content/drive.")
except Exception as e:
    print(f"ERRO ao montar o Google Drive: {e}", file=sys.stderr)

# --- Define Caminhos (AJUSTE SE NECESSÁRIO) ---
# (Como estava antes)
DRIVE_BASE_PATH = "/content/drive/My Drive/Colab Notebooks" # Exemplo, ajuste!
CV_JSON_PATH = os.path.join(DRIVE_BASE_PATH, "datasets/dataset-cv-leonardo.json")
# **Mudar o nome do arquivo de índice para refletir a mudança (opcional, mas recomendado)**
FAISS_INDEX_PATH = os.path.join(DRIVE_BASE_PATH, "index/faiss-cv-leonardo-index-IP.bin") # <-- MUDOU O NOME

# --- Variáveis para os dados e índice ---
# (Como estava antes)
cv_data = None
cv_descriptions = None
faiss_index = None

# --- Carrega o Dataset (CV JSON) ---
# (Como estava antes - carregar cv_data e cv_descriptions)
if os.path.exists(CV_JSON_PATH):
    try:
        with open(CV_JSON_PATH, 'r', encoding='utf-8') as f:
            cv_data = json.load(f)
        print(f"Dataset CV carregado de: {CV_JSON_PATH}")
        cv_descriptions = [entry.get('Description', '') for entry in cv_data]
        print(f"Extraídas {len(cv_descriptions)} descrições do CV.")
        if not cv_descriptions:
             print("AVISO: Nenhuma descrição encontrada no arquivo JSON.", file=sys.stderr)
             cv_data = None
    except json.JSONDecodeError:
        print(f"ERRO: O arquivo {CV_JSON_PATH} não é um JSON válido.", file=sys.stderr)
    except Exception as e:
        print(f"ERRO ao carregar ou processar o arquivo JSON do CV: {e}", file=sys.stderr)
else:
    print(f"ERRO: Arquivo do dataset CV não encontrado em: {CV_JSON_PATH}. Verifique o caminho.", file=sys.stderr)


# --- Cria/Carrega Embeddings e Índice FAISS (usando IndexFlatIP) ---
if cv_data and embedder_model:
    try:
        # Verifica se o índice FAISS (versão IP) já existe
        if os.path.exists(FAISS_INDEX_PATH):
            print(f"Tentando carregar índice FAISS existente (IP) de: {FAISS_INDEX_PATH}")
            faiss_index = faiss.read_index(FAISS_INDEX_PATH)
            # Validações (como antes)
            if faiss_index.d != embedder_model.get_sentence_embedding_dimension():
                 print(f"AVISO: Dimensão do índice ({faiss_index.d}) diferente. Recriando.", file=sys.stderr)
                 faiss_index = None
            elif faiss_index.ntotal != len(cv_descriptions):
                 print(f"AVISO: Número de vetores ({faiss_index.ntotal}) diferente. Recriando.", file=sys.stderr)
                 faiss_index = None
            else:
                 print(f"Índice FAISS (IP) carregado com sucesso ({faiss_index.ntotal} vetores).")

        # Se não carregou, cria um novo
        if faiss_index is None:
            print("Gerando embeddings para as descrições do CV...")
            start_time = time.time()
            embeddings = embedder_model.encode(cv_descriptions, show_progress_bar=True)
            embeddings = np.array(embeddings).astype('float32')

            # **** NORMALIZAÇÃO DOS EMBEDDINGS ****
            print("Normalizando embeddings (L2 norm)...")
            faiss.normalize_L2(embeddings)
            # *************************************

            dimension = embeddings.shape[1]
            print(f"Criando novo índice FAISS com IndexFlatIP (Similaridade Cosseno), dimensão {dimension}...")
            # **** USAR IndexFlatIP ****
            faiss_index = faiss.IndexFlatIP(dimension)
            # **************************
            faiss_index.add(embeddings) # Adiciona os embeddings NORMALIZADOS
            print(f"Índice FAISS (IP) criado e populado com {faiss_index.ntotal} vetores.")

            # Salva o novo índice (IP)
            try:
                print(f"Salvando índice FAISS (IP) em: {FAISS_INDEX_PATH}")
                os.makedirs(os.path.dirname(FAISS_INDEX_PATH), exist_ok=True)
                faiss.write_index(faiss_index, FAISS_INDEX_PATH)
                print("Índice FAISS (IP) salvo com sucesso.")
            except Exception as e:
                print(f"ERRO ao salvar o índice FAISS (IP): {e}", file=sys.stderr)

    except Exception as e:
        print(f"ERRO durante a geração de embeddings ou manipulação do índice FAISS (IP): {e}", file=sys.stderr)
        faiss_index = None

elif not embedder_model:
     print("AVISO: Processo de indexação pulado (modelo de embedding não carregado).", file=sys.stderr)
else:
     print("AVISO: Processo de indexação pulado (dados do CV não carregados).", file=sys.stderr)

Mounted at /content/drive
Google Drive montado com sucesso em /content/drive.
Dataset CV carregado de: /content/drive/My Drive/Colab Notebooks/datasets/dataset-cv-leonardo.json
Extraídas 27 descrições do CV.
Gerando embeddings para as descrições do CV...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Normalizando embeddings (L2 norm)...
Criando novo índice FAISS com IndexFlatIP (Similaridade Cosseno), dimensão 384...
Índice FAISS (IP) criado e populado com 27 vetores.
Salvando índice FAISS (IP) em: /content/drive/My Drive/Colab Notebooks/index/faiss-cv-leonardo-index-IP.bin
Índice FAISS (IP) salvo com sucesso.


In [ ]:
# ===================================================
#             CÉLULA 5: FUNÇÃO RAG PRINCIPAL (COM MAIS DEBUG)
# ===================================================

def ask_cv_assistant(query, k=3, similarity_threshold=0.45): # <-- Ajustado threshold para Cosine
    """
    Realiza uma consulta RAG no CV carregado.
    USA SIMILARIDADE COSSENO (requer IndexFlatIP e normalização - veja Célula 4 modificada)

    Args:
        query (str): A pergunta do usuário sobre o CV.
        k (int): Número máximo de seções relevantes a recuperar.
        similarity_threshold (float): Limiar de similaridade cosseno (0 a 1).
                                      Valores mais altos indicam maior similaridade.

    Returns:
        tuple: (str: resposta_gerada, list: documentos_recuperados)
               Retorna mensagens de erro na string de resposta em caso de falha.
    """
    # --- Verificações Iniciais ---
    # (Mantenha as verificações como estavam)
    if not chat_model: return "ERRO: O modelo Gemini não está pronto.", []
    if not embedder_model: return "ERRO: O modelo de embedding não está pronto.", []
    if not faiss_index: return "ERRO: O índice FAISS (CV) não está pronto.", []
    if not cv_data: return "ERRO: Os dados do CV não foram carregados.", []

    print(f"\n[RAG] Processando consulta: '{query}'")

    try:
        # --- 1. Codificar a Consulta e NORMALIZAR ---
        start_time = time.time()
        query_embedding = embedder_model.encode([query])
        query_embedding = np.array(query_embedding).astype('float32')
        faiss.normalize_L2(query_embedding) # Normaliza para usar com IndexFlatIP (Cosine Similarity)
        print(f"[RAG] Consulta codificada e normalizada em {time.time() - start_time:.3f} seg.")

        # --- 2. Buscar no FAISS (usando IndexFlatIP) ---
        start_time = time.time()
        # search retorna similaridades (Inner Product/Cosine) e índices
        similarities, indices = faiss_index.search(query_embedding, k)
        print(f"[RAG] Busca FAISS concluída em {time.time() - start_time:.3f} seg. Encontrados {len(indices[0])} vizinhos.")

        # --- 3. Recuperar e Filtrar Documentos (usando similaridade) ---
        retrieved_docs_info = []
        relevant_indices_found = [] # Para evitar duplicatas se k for grande
        if len(indices[0]) > 0 and indices[0][0] != -1:
             for i, idx in enumerate(indices[0]):
                 if idx < len(cv_data) and idx not in relevant_indices_found: # Checagem de segurança e duplicatas
                      doc = cv_data[idx]
                      similarity = similarities[0][i] # Similaridade Cosseno

                      # **** DEBUG: Imprimir o documento recuperado ANTES de filtrar ****
                      print(f"\n  [DEBUG] Recuperado Doc Índice: {idx}, Similaridade: {similarity:.4f}")
                      print(f"  [DEBUG] Categoria: {doc.get('Category', 'N/A')}, Sub: {doc.get('Subcategory', 'N/A')}")
                      print(f"  [DEBUG] Descrição: {doc.get('Description', '')[:300]}...") # Primeiros 300 chars
                      # ****************************************************************

                      # Filtrar por similaridade
                      if similarity >= similarity_threshold:
                          retrieved_docs_info.append({
                              "id": doc.get("ID", f"index_{idx}"),
                              "category": doc.get("Category", "N/A"),
                              "subcategory": doc.get("Subcategory", "N/A"),
                              "description": doc.get("Description", ""),
                              "similarity": similarity # Armazenar similaridade em vez de distância
                          })
                          relevant_indices_found.append(idx)
                          print(f"    -> Documento {idx} INCLUÍDO (Similaridade {similarity:.4f} >= {similarity_threshold})")
                      else:
                          print(f"    -> Documento {idx} DESCARTADO (Similaridade {similarity:.4f} < {similarity_threshold})")

                 elif idx in relevant_indices_found:
                      print(f"  [DEBUG] Índice {idx} já processado, pulando.")
                 else:
                      print(f"  [DEBUG] AVISO: Índice FAISS {idx} inválido retornado.", file=sys.stderr)

        if not retrieved_docs_info:
            print("[RAG] Nenhum documento relevante encontrado (ou todos abaixo do threshold) no CV para esta consulta.")
            # Tenta dar uma resposta mais útil se NADA foi recuperado
            if len(indices[0]) > 0 and indices[0][0] != -1:
                 # Pega o mais similar mesmo que abaixo do threshold para dar um feedback
                 top_idx = indices[0][0]
                 top_sim = similarities[0][0]
                 top_doc_desc = cv_data[top_idx].get('Description', '')[:200]
                 return (f"Não encontrei informações altamente relevantes (acima do limiar de {similarity_threshold*100:.0f}% de similaridade) para isso no CV. "
                         f"A seção mais próxima encontrada (similaridade {top_sim:.2f}) foi sobre: '{top_doc_desc}...', mas pode não ser o que você procura."), []
            else:
                 return "Não encontrei nenhuma seção no CV que pareça relacionada a sua pergunta.", []

        print(f"[RAG] Documentos relevantes (acima do threshold) selecionados: {len(retrieved_docs_info)}")

        # --- 4. Construir Contexto para o LLM ---
        context_parts = []
        for i, doc_info in enumerate(retrieved_docs_info):
            # Deixar mais claro para o LLM de onde vem a informação
            context_parts.append(f"--- [INÍCIO Seção Relevante {i+1} do CV - ID: {doc_info['id']}, Similaridade: {doc_info['similarity']:.2f}] ---\n"
                                 f"{doc_info['description']}\n"
                                 f"--- [FIM Seção Relevante {i+1}] ---")
        context = "\n\n".join(context_parts)

        # **** DEBUG: Imprimir o contexto final que vai para o LLM ****
        print("\n" + "="*20 + " CONTEÚDO ENVIADO PARA O GEMINI " + "="*20)
        print(f"[DEBUG] Contexto Construído ({len(context)} caracteres):\n{context}")
        print("="*60)
        # ***********************************************************

        # --- 5. Preparar Mensagens para Gemini (Prompt Ajustado) ---
        system_prompt = (
            "Você é um assistente de IA para análise de currículos (CVs). "
            "Sua tarefa é responder à pergunta do usuário baseando-se *prioritariamente* nas informações das seções do CV fornecidas entre [INÍCIO Seção Relevante] e [FIM Seção Relevante]. "
            "Sintetize a informação encontrada nessas seções para fornecer uma resposta coesa. "
            "Se a resposta estiver claramente presente, responda diretamente. "
            "Se a informação não estiver nas seções fornecidas, ou se as seções não forem relevantes para a pergunta, informe que a resposta não foi encontrada *nesse contexto específico do CV*. "
            #"Não adicione conhecimento externo."
            "Caso precise, utilize informação externa para trazer uma resposta mais completa, mas sem ."
            "A sessão de description é o foco das informações."
            # "Mencione brevemente as seções do CV (por ID ou categoria) que você usou para basear sua resposta, se possível." # Instrução opcional
        )
        human_prompt = (
            f"**Contexto Extraído do Currículo:**\n"
            f"{context}\n\n"
            f"**Pergunta do Usuário:**\n{query}\n\n"
            f"**Instrução Final:** Use o contexto acima para responder à pergunta."
        )

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=human_prompt)
        ]

        # --- 6. Invocar o Modelo Gemini ---
        print("[RAG] Enviando consulta para o modelo Gemini...")
        start_time = time.time()
        response = chat_model.invoke(messages)
        generation_time = time.time() - start_time
        print(f"[RAG] Resposta recebida do Gemini em {generation_time:.2f} seg.")

        # --- 7. Retornar Resultado ---
        return response.content, retrieved_docs_info

    except Exception as e:
        # (Manter o tratamento de erro como estava)
        print(f"ERRO GERAL durante a execução da consulta RAG: {e}", file=sys.stderr)
        retrieved = retrieved_docs_info if 'retrieved_docs_info' in locals() else []
        return f"Ocorreu um erro inesperado ao processar sua pergunta: {e}", retrieved

In [ ]:
# CÉLULA 6: INTERAÇÃO COM O USUÁRIO


# Verifica se tudo está pronto para começar
if chat_model and embedder_model and faiss_index and cv_data:
    print("\n===================================")
    print(" Assistente de Análise de CV pronto! ")
    print("===================================")
    print("Digite sua pergunta sobre o currículo carregado ou 'sair' para terminar.")

    while True:
        user_query = input("\nSua pergunta: ")
        if user_query.lower().strip() == 'sair':
            print("Encerrando o assistente. Até logo!")
            break
        if not user_query.strip():
            continue

        # Chama a função RAG principal
        answer, sources = ask_cv_assistant(user_query, k=4) # Pega os 4 mais relevantes

        print("\n--------------------")
        print("Resposta do Assistente:")
        print(answer)
        print("--------------------")

        if sources:
             print("\nFontes utilizadas (Seções do CV):")
             for src in sources:
                  print(f"  - ID: {src['id']} (Distância: {src['distance']:.4f}) - {src['description'][:100]}...") # Mostra início da descrição
        print("--------------------\n")

else:
    print("\nERRO CRÍTICO: O assistente não pode ser iniciado.", file=sys.stderr)
    if not chat_model: print("- Modelo Gemini não inicializado.", file=sys.stderr)
    if not embedder_model: print("- Modelo de embedding não inicializado.", file=sys.stderr)
    if not faiss_index: print("- Índice FAISS não inicializado/carregado.", file=sys.stderr)
    if not cv_data: print("- Dados do CV não carregados.", file=sys.stderr)
    print("Verifique as mensagens de erro nas células anteriores, especialmente a configuração da API Key, caminhos de arquivos e montagem do Drive.", file=sys.stderr)



 Assistente de Análise de CV pronto! 
Digite sua pergunta sobre o currículo carregado ou 'sair' para terminar.

Sua pergunta: Me traga informações desse curriculo realcionadas a React

[RAG] Processando consulta: 'Me traga informações desse curriculo realcionadas a React'
[RAG] Consulta codificada e normalizada em 0.025 seg.
[RAG] Busca FAISS concluída em 0.000 seg. Encontrados 4 vizinhos.

  [DEBUG] Recuperado Doc Índice: 2, Similaridade: 0.4934
  [DEBUG] Categoria: Skills, Sub: Technical
  [DEBUG] Descrição: React.js...
    -> Documento 2 INCLUÍDO (Similaridade 0.4934 >= 0.45)

  [DEBUG] Recuperado Doc Índice: 16, Similaridade: 0.3142
  [DEBUG] Categoria: Experience, Sub: Engenheiro Mecânico
  [DEBUG] Descrição: Callidus Engenharia, Mechanical Engineer (March 2021 - May 2022). Conducted safety inspections (NR-13, NR-12, NR-10), designed HVAC systems (climate control, refrigeration, air renewal), prepared technical reports, and worked on industrial projects (electrical, hydraulic, me

KeyError: 'distance'